<a href="https://colab.research.google.com/github/parismoscow/Alzheimer-s_Disease/blob/master/Alzheimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imblearn

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from collections import Counter




/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# df = pd.read_csv("/content/drive/My Drive/measures.csv")
# df = pd.read_csv("/content/drive/My Drive/pct_changes.csv")
# choose either 'DX' or 'DXCHANGE' for prediction
# df = pd.read_csv("/content/drive/My Drive/TADPOLE_D1_D2.csv").loc[:, ["PTID", "DX", "PTEDUCAT", "ADAS13", "Ventricles", "CDRSB", "ADAS11", "MMSE",  "RAVLT_immediate", "Hippocampus", "WholeBrain", "Entorhinal", "MidTemp"]]
df = pd.read_csv("/content/drive/My Drive/TADPOLE_D1_D2.csv").loc[:, ["PTID", "DX", "PTGENDER", "PTEDUCAT", "ADAS13", "Ventricles", "CDRSB", "ADAS11", "MMSE",  "RAVLT_immediate", "Hippocampus", "WholeBrain", "Entorhinal", "MidTemp", "FDG", "AV45", "ABETA_UPENNBIOMK9_04_19_17", "TAU_UPENNBIOMK9_04_19_17", "PTAU_UPENNBIOMK9_04_19_17", "APOE4"]]
# df = pd.read_csv("/content/drive/My Drive/TADPOLE_D1_D2.csv").loc[:, ["PTID", "DXCHANGE", "PTRACCAT", "PTETHCAT", "PTGENDER", "PTEDUCAT", "CDRSB", "ADAS11", "MMSE", "ADAS13", "RAVLT_immediate"]]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724

In [5]:
df.shape[0]

12741

In [0]:
# df['DX'].value_counts()

Use all suggested features

In [0]:
def cleanup_data (df, drop):
  df = df.replace(r'^\s*$', np.nan, regex=True)
#   df.fillna(value=0, inplace=True)
  if drop:
    df.dropna(inplace=True)
  else:
    df.fillna(value=0, inplace=True)
  
  
  if 'ABETA_UPENNBIOMK9_04_19_17' in df: 
    df.loc[df['ABETA_UPENNBIOMK9_04_19_17'] == '<200', 'ABETA_UPENNBIOMK9_04_19_17'] = 199
    df['ABETA_UPENNBIOMK9_04_19_17'] = df['ABETA_UPENNBIOMK9_04_19_17'].astype(float)
  if 'TAU_UPENNBIOMK9_04_19_17' in df: 
    df.loc[df['TAU_UPENNBIOMK9_04_19_17'] == '>1300', 'TAU_UPENNBIOMK9_04_19_17'] = 1301
    df.loc[df['TAU_UPENNBIOMK9_04_19_17'] == '<80', 'TAU_UPENNBIOMK9_04_19_17'] = 79
    df['TAU_UPENNBIOMK9_04_19_17'] = df['TAU_UPENNBIOMK9_04_19_17'].astype(float)
  if 'PTAU_UPENNBIOMK9_04_19_17' in df:
    df.loc[df['PTAU_UPENNBIOMK9_04_19_17'] == '>120', 'PTAU_UPENNBIOMK9_04_19_17'] = 121
    df.loc[df['PTAU_UPENNBIOMK9_04_19_17'] == '<8', 'PTAU_UPENNBIOMK9_04_19_17'] = 7
    df['PTAU_UPENNBIOMK9_04_19_17'] = df['PTAU_UPENNBIOMK9_04_19_17'].astype(float)


  # Remove diagnosis changes that are unlikely
#   df = df.loc[(df['final_DX'] == 'MCI') | (df['final_DX'] == 'Dementia') | (df['final_DX'] == 'NL')]
  df = df.loc[(df['DX'] == 'MCI') | (df['DX'] == 'Dementia') | (df['DX'] == 'NL')]

#   df = df.loc[(df['DXCHANGE'] == 1) | (df['DXCHANGE'] == 2) | (df['DXCHANGE'] == 3) | (df['DXCHANGE'] == 4) | (df['DXCHANGE'] == 5)]
  
  
  return df

In [0]:
drop = True
df = cleanup_data(df, drop)

In [0]:
def add_pct_change(df, feature_list):
  for feature in feature_list:
    new_column = "pct_" + feature
#     df[feature].loc[df[feature] == 0][feature] = 1
#     if divisor != 0:
    df[new_column] = 100 * (1 - df.iloc[0][feature] / df[feature])
  return df



In [0]:
def add_sum_column (df):
  df['pct_sum'] = df['pct_Hippocampus'] + df['pct_WholeBrain'] + df['pct_Entorhinal'] + df['pct_MidTemp']
  return df

In [0]:
def add_final_dx_column(df):
  for dx in ['Dementia', 'MCI', "NL"]:
    if dx in df['DX'].values:
      df['final_DX'] = dx 
      return df

In [0]:
# df = df.groupby('PTID').apply(add_final_dx_column)


In [0]:
df = df.groupby('PTID').apply(add_pct_change, ['Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'MidTemp'])
df = df.groupby('PTID').apply(add_sum_column)
# df = df.groupby('PTID').apply(add_final_dx_column)
# df = df.groupby('PTID').apply(add_pct_change, ["ADAS13", "CDRSB", "ADAS11", "MMSE", "RAVLT_immediate"])


In [13]:
df.shape[0]

813

In [0]:
def show_data (df, category):
  print ("There are ", df[category].count(), " records")
  print ("Category breakdown \n", df[category].value_counts())

In [15]:
show_data(df, 'DX')

There are  813  records
Category breakdown 
 MCI         425
NL          281
Dementia    107
Name: DX, dtype: int64


In [0]:
def populate_X_y (df, y_cat, encode_list):
  y = df[y_cat]
  X = df.drop(columns=[y_cat, 'PTID'])
#   X = df.drop(columns=[y_cat, 'PTID'])
# todo replace following line with one above
#   X = df.drop(columns=[y_cat])
  label_encoder = LabelEncoder()
  for label in encode_list:
    if label in X:
      encode = X[label]
      label_encoder.fit(encode)
      X[label]= label_encoder.transform(encode)
  return X, y

In [0]:
X, y = populate_X_y (df, 'DX', ["PTRACCAT", "PTETHCAT", "PTGENDER"])

Split data into train and test

In [0]:
X_train, X_test, y_train, y_test = train_test_split (X, y, random_state=42)

PCA

In [0]:
def myPCA (X_train, X_test):
  pca = PCA()  
  X_train = pca.fit_transform(X_train)  
  X_test = pca.transform(X_test)  
  return X_train, X_test

In [0]:
# X_train, X_test = myPCA (X_train, X_test)

Scale the features

In [0]:
def scale_features (scaler, X_train, X_test):
  if scaler == 'minmax':
    X_train = MinMaxScaler().fit(X_train).transform(X_train);
    X_test = MinMaxScaler().fit(X_test).transform(X_test);
  elif scaler == 'standard':
    X_train = StandardScaler().fit(X_train).transform(X_train);
    X_test = StandardScaler().fit(X_test).transform(X_test);
    
  return X_train, X_test
  

In [0]:
# X_test.dtypes

In [0]:
X_train, X_test = scale_features ('standard', X_train, X_test)

In [0]:
# X_train.head(50)

In [0]:
def oversample (alg, X_train, y_train):
  if alg == 'smote':
    smt = SMOTE()
    X_train, y_train = smt.fit_sample(X_train, y_train)
    print('Resampled dataset shape %s' % Counter(y_train))
    return X_train, y_train
  if alg == 'random':
    ros = RandomOverSampler(random_state=42)
    X_train, y_train = ros.fit_sample(X_train, y_train)
    print('Resampled dataset shape %s' % Counter(y_train))
    return X_train, y_train

In [26]:
# oversample using SMOTE or random
X_train, y_train = oversample ('smote', X_train, y_train)

Resampled dataset shape Counter({'MCI': 317, 'NL': 317, 'Dementia': 317})


Select and run a model

In [0]:
def run_model (model_name, X_train, X_test, y_train, y_test):
  if 'svc' in model_name.lower():
    model = SVC(kernel='linear')
    
  elif 'random forest' in model_name.lower():
    model = RandomForestClassifier(n_estimators=200)
  
  elif 'logistic regression' in model_name.lower():
    model = LogisticRegression()
      
  model.fit(X_train, y_train)
  print ("*************", model_name, "*************\n", "\nScore is: ", model.score(X_test, y_test), "\n")
  predictions = model.predict(X_test)
  print ("\nConfusion matrix\n", confusion_matrix(y_test, predictions))
  print("\n", classification_report(y_test, predictions))
#   print("\n", classification_report(y_test, predictions, target_names=["NL to NL", "MCI to MCI", "AD to AD", "NL to MCI", "MCI to AD" ]))

  return model


In [28]:
y_test.value_counts()

MCI         108
NL           71
Dementia     25
Name: DX, dtype: int64

In [37]:
model = run_model('random forest with prct change, new sum column', X_train, X_test, y_train, y_test)

************* random forest with prct change, new sum column *************
 
Score is:  0.9313725490196079 


Confusion matrix
 [[ 22   3   0]
 [  5 101   2]
 [  0   4  67]]

               precision    recall  f1-score   support

    Dementia       0.81      0.88      0.85        25
         MCI       0.94      0.94      0.94       108
          NL       0.97      0.94      0.96        71

    accuracy                           0.93       204
   macro avg       0.91      0.92      0.91       204
weighted avg       0.93      0.93      0.93       204



In [30]:
importances = model.feature_importances_
X_features = X.columns
sorted(zip(model.feature_importances_, X_features), reverse=True)



[(0.3165900470230781, 'CDRSB'),
 (0.11569776979236349, 'MMSE'),
 (0.11246923930463948, 'ADAS13'),
 (0.0967681089712002, 'ADAS11'),
 (0.059272980390757894, 'RAVLT_immediate'),
 (0.052144822639033445, 'FDG'),
 (0.035093182574552524, 'Hippocampus'),
 (0.030412840083543474, 'AV45'),
 (0.027887195420059213, 'ABETA_UPENNBIOMK9_04_19_17'),
 (0.020883249383372555, 'Entorhinal'),
 (0.020516146761139796, 'TAU_UPENNBIOMK9_04_19_17'),
 (0.02038206816301778, 'Ventricles'),
 (0.018147171699403664, 'PTAU_UPENNBIOMK9_04_19_17'),
 (0.0175413705654917, 'WholeBrain'),
 (0.016693550110949685, 'MidTemp'),
 (0.01050933365875073, 'PTEDUCAT'),
 (0.009888982884200039, 'APOE4'),
 (0.0039912433794230566, 'PTGENDER'),
 (0.003052634970240467, 'pct_MidTemp'),
 (0.0028031096142165568, 'pct_Entorhinal'),
 (0.002477648371165635, 'pct_Hippocampus'),
 (0.002366628664697332, 'pct_sum'),
 (0.0022357066838253857, 'pct_WholeBrain'),
 (0.002174968890877967, 'pct_Ventricles')]

In [31]:
len(X_features)

24

DNN


In [32]:
from keras.models import Sequential
from keras.layers import Dense

def build_NN_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, activation='relu', input_dim=24))
    classifier.add(Dense(units=6, activation='relu'))
    classifier.add(Dense(units=3, activation='softmax'))
    
#     classifier.add(Dense(units=2, activation='relu', input_dim=22))
#     classifier.add(Dense(units=3, activation='softmax'))
    # classifier.summary()
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier

# Create a Keras model that's compatible with scikit-learn
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
keras_NN_classifier = KerasClassifier(build_NN_classifier, epochs=300, shuffle=True, verbose=2, callbacks=[EarlyStopping(monitor='acc', patience=20, verbose=2)])

Using TensorFlow backend.


In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# from imblearn.over_sampling import SMOTE

# pca = PCA()
# smt = SMOTE(random_state=42)

# deep_model = make_pipeline(StandardScaler(), smt, pca, keras_NN_classifier)
deep_model = make_pipeline(StandardScaler(), keras_NN_classifier)

In [34]:
deep_model.fit(X_train, y_train)

W0630 05:25:26.348695 139931325728640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0630 05:25:26.366319 139931325728640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0630 05:25:26.368776 139931325728640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0630 05:25:26.413208 139931325728640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0630 05:25:26.436387 139931325728640 deprecation_wrappe

Epoch 1/300
 - 1s - loss: 1.0266 - acc: 0.5247
Epoch 2/300
 - 0s - loss: 0.8873 - acc: 0.5931
Epoch 3/300
 - 0s - loss: 0.7991 - acc: 0.6278
Epoch 4/300
 - 0s - loss: 0.7439 - acc: 0.6530
Epoch 5/300
 - 0s - loss: 0.7024 - acc: 0.6677
Epoch 6/300
 - 0s - loss: 0.6693 - acc: 0.6814
Epoch 7/300
 - 0s - loss: 0.6413 - acc: 0.6909
Epoch 8/300
 - 0s - loss: 0.6182 - acc: 0.7087
Epoch 9/300
 - 0s - loss: 0.5988 - acc: 0.7150
Epoch 10/300
 - 0s - loss: 0.5821 - acc: 0.7182
Epoch 11/300
 - 0s - loss: 0.5678 - acc: 0.7234
Epoch 12/300
 - 0s - loss: 0.5547 - acc: 0.7319
Epoch 13/300
 - 0s - loss: 0.5422 - acc: 0.7350
Epoch 14/300
 - 0s - loss: 0.5312 - acc: 0.7424
Epoch 15/300
 - 0s - loss: 0.5211 - acc: 0.7466
Epoch 16/300
 - 0s - loss: 0.5128 - acc: 0.7476
Epoch 17/300
 - 0s - loss: 0.5042 - acc: 0.7550
Epoch 18/300
 - 0s - loss: 0.4968 - acc: 0.7634
Epoch 19/300
 - 0s - loss: 0.4888 - acc: 0.7708
Epoch 20/300
 - 0s - loss: 0.4820 - acc: 0.7771
Epoch 21/300
 - 0s - loss: 0.4759 - acc: 0.7729
E

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('kerasclassifier',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f441b4d1550>)],
         verbose=False)

In [35]:
model_accuracy = deep_model.score(X_test, y_test)
print("Deep Neural Network - Accuracy: {}".format(model_accuracy))

Deep Neural Network - Accuracy: 0.8921568639138165
